<a href="https://colab.research.google.com/github/windylobster217-ux/face-imagerecognisation/blob/main/Vmsfaceimagedetction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# VISITOR RECOGNITION ASSET GENERATION (FINAL VERSION WITH CV2 LOADING FIX)
# ==============================================================================

# -----------------
# 1. SETUP & CONFIG (Assuming libraries are installed and drive is mounted)
# -----------------
import torch
import numpy as np
import pickle
import os
import sys
import cv2 # <-- NEW: Import OpenCV
from PIL import Image
from torchvision import datasets, transforms
import torch.onnx
from facenet_pytorch import MTCNN, InceptionResnetV1
from google.colab import drive

# Define file paths
DATA_DIR = '/content/drive/MyDrive/IMAGE DATASET'
EMBEDDINGS_FILE = '/content/drive/MyDrive/visitor_embeddings.pkl'
ONNX_MODEL_PATH = '/content/drive/MyDrive/facenet_model.onnx'

# Re-mount Drive and select device (assumes successful prior installation)
try:
    drive.mount('/content/drive', force_remount=True)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
except Exception:
    device = torch.device('cpu')

# --- Custom Image Loading Function to bypass PIL/Dtype issues ---
def load_and_convert_to_pil(image_path):
    # Use cv2.imread for robust file handling
    img_np = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img_np is None:
        raise ValueError("cv2.imread failed to load the image.")
    # Convert from BGR to RGB
    img_np = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
    # Convert to PIL Image for MTCNN compatibility
    return Image.fromarray(img_np)

# --------------------------
# 2. PIPELINE STEP: FILE CLEANUP AND IMAGE CONVERSION (Standardization)
# --------------------------
print("\n--- 2. File Cleanup & Aggressive Image Standardization ---")
valid_image_count = 0

for root, _, files in os.walk(DATA_DIR):
    for filename in files:
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            old_path = os.path.join(root, filename)
            # Normalize filename (re-run for safety)
            new_filename = filename.replace(' ', '_').replace('#', '').replace(' ', '')
            new_path = os.path.join(root, new_filename)
            if old_path != new_path:
                os.rename(old_path, new_path)

            # Aggressive Format Conversion (resaving as standard RGB JPEG)
            try:
                img = Image.open(new_path)
                if img.mode != 'RGB': img = img.convert('RGB')
                img.save(new_path, 'JPEG')
                valid_image_count += 1
            except Exception as e:
                print(f"❌ Corrupted/Unreadable file detected: {new_path}. Error: {e}")
                os.rename(new_path, new_path + '.corrupt')

if valid_image_count == 0:
    print("\n❌ CRITICAL ERROR: NO valid image files found after conversion. ABORTING.")
    sys.exit(1)

print(f"✅ Finished image standardization. {valid_image_count} files processed successfully.")


# --------------------------
# 3. MODELS INITIALIZATION
# --------------------------
print("\n--- 3. Models Initialization ---")
mtcnn = MTCNN(image_size=160, margin=14, min_face_size=20, post_process=True, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
print('✅ Models initialized.')

# --------------------------
# 4. PIPELINE STEP: GENERATE EMBEDDINGS (THE CORE PROCESS)
# --------------------------
print("\n--- 4. Generating Embeddings (Using Manual CV2 Load) ---")
all_image_paths = []
all_labels = []

# Manually collect paths and labels
for root, dirs, files in os.walk(DATA_DIR):
    if root != DATA_DIR: # Skip the root folder
        label_name = os.path.basename(root)
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')) and not file.endswith('.corrupt'):
                all_image_paths.append(os.path.join(root, file))
                all_labels.append(label_name)

embeddings_raw = {label: [] for label in set(all_labels)}
success_count = 0

with torch.no_grad():
    for img_path, name in zip(all_image_paths, all_labels):
        try:
            # LOAD IMAGE with robust CV2 method
            img = load_and_convert_to_pil(img_path)

            face_tensor = mtcnn(img)
            if face_tensor is not None:
                face_tensor_batch = face_tensor.to(device).unsqueeze(0)
                embedding = resnet(face_tensor_batch).cpu().numpy()[0]

                embeddings_raw[name].append(embedding)
                success_count += 1
            else:
                print(f'⚠️ Warning: No face detected for {img_path}. Skipping.')
        except Exception as e:
            # If it fails here, the file is unreadable even by CV2
            print(f'❌ CRITICAL FINAL FAILURE processing {img_path}: {e}')

print(f'\n✅ Finished generating raw embeddings. Total successful: {success_count}')
if success_count == 0:
    print("❌ Aborting export as no embeddings were created.")
    sys.exit(1)

# --------------------------
# 5. PIPELINE STEP: CREATE & SAVE VISITOR DATABASE (PKL)
# --------------------------
visitor_database = {}
for name, emb_list in embeddings_raw.items():
    if len(emb_list) > 0:
        mean_embedding = np.mean(emb_list, axis=0)
        visitor_database[name] = mean_embedding / np.linalg.norm(mean_embedding)

try:
    with open(EMBEDDINGS_FILE, 'wb') as f:
        pickle.dump(visitor_database, f)
    print(f'\n--- 5. Final Visitor Database (PKL) ---')
    print(f'Database saved to: {EMBEDDINGS_FILE}')
except Exception as e:
    print(f'❌ ERROR: Could not save PKL file to Drive: {e}')


# --------------------------
# 6. PIPELINE STEP: EXPORT MODEL TO ONNX
# --------------------------
resnet.eval().cpu()
dummy_input = torch.randn(1, 3, 160, 160)

try:
    torch.onnx.export(resnet, dummy_input, ONNX_MODEL_PATH, opset_version=12, do_constant_folding=True,
                      input_names=['input'], output_names=['output'],
                      dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})
    print(f'\n--- 6. ONNX Model Export ---')
    print(f'✅ Model exported to ONNX: {ONNX_MODEL_PATH}')
except Exception as e:
    print(f'❌ CRITICAL ERROR: Failed to export ONNX model: {e}')


Mounted at /content/drive

--- 2. File Cleanup & Aggressive Image Standardization ---
✅ Finished image standardization. 20 files processed successfully.

--- 3. Models Initialization ---
✅ Models initialized.

--- 4. Generating Embeddings (Using Manual CV2 Load) ---
❌ CRITICAL FINAL FAILURE processing /content/drive/MyDrive/IMAGE DATASET/yuva images/Photo_on_24-10-25_at_1.38PM_2.jpg: module 'cv2' has no attribute 'imread'
❌ CRITICAL FINAL FAILURE processing /content/drive/MyDrive/IMAGE DATASET/yuva images/Photo_on_24-10-25_at_1.38PM_3.jpg: module 'cv2' has no attribute 'imread'
❌ CRITICAL FINAL FAILURE processing /content/drive/MyDrive/IMAGE DATASET/yuva images/Photo_on_24-10-25_at_1.38PM_4.jpg: module 'cv2' has no attribute 'imread'
❌ CRITICAL FINAL FAILURE processing /content/drive/MyDrive/IMAGE DATASET/yuva images/Photo_on_24-10-25_at_1.38PM_5.jpg: module 'cv2' has no attribute 'imread'
❌ CRITICAL FINAL FAILURE processing /content/drive/MyDrive/IMAGE DATASET/yuva images/Photo_on_24-

SystemExit: 1

In [ ]:
# ==============================================================================
# CELL 1: FINAL INSTALLATION & CV2 VERIFICATION
# ==============================================================================
import time
import os
import sys
import subprocess
from IPython import get_ipython

print("--- Starting Final Dependency Check and Reinstallation ---")

# 1. Aggressively uninstall EVERYTHING to clear paths
print("Clearing all conflicting packages...")
packages_to_uninstall = [
    "torch", "torchvision", "torchaudio", "numpy", "tensorflow",
    "numba", "cupy-cuda12x", "opencv-python", "opencv-contrib-python", "opencv-python-headless", "facenet-pytorch", "onnx"
]
# We pipe output to /dev/null to hide noisy "Not Installed" messages
subprocess.check_call([sys.executable, "-m", "pip", "uninstall"] + packages_to_uninstall + ["-y"],
                      stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 2. Install STABLE, functional versions
print("Installing core dependencies (torch, numpy, opencv-python, facenet-pytorch)...")
# NumPy must be installed first!
subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy==1.26.4", "--quiet"])
subprocess.check_call([sys.executable, "-m", "pip", "install",
                      "torch==2.2.2", "torchvision==0.17.2", "torchaudio==2.2.2",
                      "--extra-index-url", "https://download.pytorch.org/whl/cu121",
                      "--quiet"])

# CRITICAL CHANGE: Install standard opencv-python, NOT headless
subprocess.check_call([sys.executable, "-m", "pip", "install", "opencv-python", "Pillow", "--quiet"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "facenet-pytorch==2.5.3", "onnx", "--quiet"])

print("\nInstallation complete.")

# 3. Handle Colab Runtime Restart
print("Restarting runtime is CRITICAL to load the new CV2 library...")
time.sleep(3)
os.kill(os.getpid(), 9)

print("Please wait for the runtime to restart before running the main script cell.")

--- Starting Final Dependency Check and Reinstallation ---
Clearing all conflicting packages...
Installing core dependencies (torch, numpy, opencv-python, facenet-pytorch)...

Installation complete.
Restarting runtime is CRITICAL to load the new CV2 library...


In [ ]:
# ==============================================================================
# FINAL GUARANTEED PIPELINE SCRIPT (ALL-IN-ONE ROBUST EXECUTION)
# ==============================================================================
import subprocess
import sys
import os

# --- 1. Aggressive Dependency Management and Installation ---
print("--- 1. Starting Aggressive Dependency Management ---")

# 1.1. Aggressively uninstall conflicting packages
print("Clearing conflicting packages...")
packages_to_uninstall = [
    "torch", "torchvision", "torchaudio", "numpy", "tensorflow",
    "opencv-python", "opencv-contrib-python", "opencv-python-headless", "facenet-pytorch", "onnx",
    "numba", "cupy-cuda12x"
]
# Use subprocess to run pip commands
try:
    subprocess.check_call([sys.executable, "-m", "pip", "uninstall"] + packages_to_uninstall + ["-y"],
                          stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
except:
    pass # Ignore errors for packages that aren't installed

# 1.2. Install guaranteed stable, functional versions
print("Installing stable core dependencies...")
# NumPy must be installed first!
subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy==1.26.4", "--quiet"])
subprocess.check_call([sys.executable, "-m", "pip", "install",
                      "torch==2.2.2", "torchvision==0.17.2", "torchaudio==2.2.2",
                      "--extra-index-url", "https://download.pytorch.org/whl/cu121",
                      "--quiet"])
# Install functional OpenCV and main libraries
subprocess.check_call([sys.executable, "-m", "pip", "install", "opencv-python", "Pillow", "--quiet"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "facenet-pytorch==2.5.3", "onnx", "--quiet"])

print("✅ Installation complete. Loading modules...")

# --- 2. Module Imports and Config ---
import torch
import numpy as np
import pickle
import cv2
from PIL import Image
from torchvision import datasets, transforms
import torch.onnx
from facenet_pytorch import MTCNN, InceptionResnetV1
from google.colab import drive

# Define file paths
DATA_DIR = '/content/drive/MyDrive/IMAGE DATASET'
EMBEDDINGS_FILE = '/content/drive/MyDrive/visitor_embeddings.pkl'
ONNX_MODEL_PATH = '/content/drive/MyDrive/facenet_model.onnx'

# Check CV2 health (this should now pass)
if not hasattr(cv2, 'imread'):
    print("❌ FATAL ERROR: CV2 is corrupted despite reinstallation. Please check Colab logs.")
    sys.exit(1)

# Mount Drive and select device
try:
    drive.mount('/content/drive', force_remount=True)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
except Exception:
    device = torch.device('cpu')


# --- Utility Functions ---
def load_and_convert_to_pil(image_path):
    # Use cv2.imread - guaranteed to be functional now
    img_np = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img_np is None:
        raise ValueError(f"CV2 failed to load image. File may be corrupted.")
    img_np = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
    return Image.fromarray(img_np)

# --------------------------
# 3. PIPELINE STEP: FILE CLEANUP AND IMAGE CONVERSION (Standardization)
# --------------------------
print("\n--- 3. File Cleanup & Aggressive Image Standardization ---")
valid_image_count = 0
current_dir_valid = os.path.isdir(DATA_DIR)

# This loop must be inside the single cell and run successfully
for root, _, files in os.walk(DATA_DIR):
    for filename in files:
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            old_path = os.path.join(root, filename)
            # 1. Filename Normalization
            new_filename = filename.replace(' ', '_').replace('#', '').replace(' ', '')
            new_path = os.path.join(root, new_filename)
            if old_path != new_path:
                os.rename(old_path, new_path)

            # 2. Aggressive Format Conversion (resaving as standard RGB JPEG via PIL)
            try:
                img = Image.open(new_path)
                if img.mode != 'RGB': img = img.convert('RGB')
                img.save(new_path, 'JPEG')
                valid_image_count += 1
            except Exception as e:
                print(f"❌ Corrupted file detected: {new_path}. Error: {e}")
                os.rename(new_path, new_path + '.corrupt')

if valid_image_count == 0:
    print("\n❌ CRITICAL ERROR: NO valid image files found after conversion. ABORTING.")
    sys.exit(1)

print(f"✅ Finished image standardization. {valid_image_count} files processed successfully.")


# --------------------------
# 4. PIPELINE STEP: MODELS INITIALIZATION
# --------------------------
print("\n--- 4. Models Initialization ---")
mtcnn = MTCNN(image_size=160, margin=14, min_face_size=20, post_process=True, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)


# --------------------------
# 5. PIPELINE STEP: GENERATE EMBEDDINGS (THE CORE PROCESS)
# --------------------------
print("\n--- 5. Generating Embeddings (Using Manual CV2 Load) ---")
all_image_paths = []
all_labels = []

# Manually collect paths and labels
for root, _, files in os.walk(DATA_DIR):
    if root != DATA_DIR:
        label_name = os.path.basename(root)
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')) and not file.endswith('.corrupt'):
                all_image_paths.append(os.path.join(root, file))
                all_labels.append(label_name)

embeddings_raw = {label: [] for label in set(all_labels)}
success_count = 0

with torch.no_grad():
    for img_path, name in zip(all_image_paths, all_labels):
        try:
            # LOAD IMAGE with robust CV2 method
            img = load_and_convert_to_pil(img_path)

            face_tensor = mtcnn(img)
            if face_tensor is not None:
                face_tensor_batch = face_tensor.to(device).unsqueeze(0)
                embedding = resnet(face_tensor_batch).cpu().numpy()[0]

                embeddings_raw[name].append(embedding)
                success_count += 1
            else:
                # Log if face detection fails (e.g., poor quality side-face image)
                print(f'⚠️ Warning: No face detected for {img_path}. Skipping.')
        except Exception as e:
            # This should only catch deep errors like actual file read failure
            print(f'❌ CRITICAL FINAL FAILURE processing {img_path}: {e}')

print(f'\n✅ Finished generating raw embeddings. Total successful: {success_count}')
if success_count == 0:
    print("❌ Aborting export as no embeddings were created.")
    sys.exit(1)

# --------------------------
# 6. PIPELINE STEP: CREATE & SAVE VISITOR DATABASE (PKL)
# --------------------------
visitor_database = {}
for name, emb_list in embeddings_raw.items():
    if len(emb_list) > 0:
        mean_embedding = np.mean(emb_list, axis=0)
        visitor_database[name] = mean_embedding / np.linalg.norm(mean_embedding)

try:
    with open(EMBEDDINGS_FILE, 'wb') as f:
        pickle.dump(visitor_database, f)
    print(f'\n--- 6. Final Visitor Database (PKL) ---')
    print(f'Database saved to: {EMBEDDINGS_FILE}')
except Exception as e:
    print(f'❌ ERROR: Could not save PKL file to Drive: {e}')


# --------------------------
# 7. PIPELINE STEP: EXPORT MODEL TO ONNX
# --------------------------
resnet.eval().cpu()
dummy_input = torch.randn(1, 3, 160, 160)

try:
    torch.onnx.export(resnet, dummy_input, ONNX_MODEL_PATH, opset_version=12, do_constant_folding=True,
                      input_names=['input'], output_names=['output'],
                      dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})
    print(f'\n--- 7. ONNX Model Export ---')
    print(f'✅ Model exported to ONNX: {ONNX_MODEL_PATH}')
except Exception as e:
    print(f'❌ CRITICAL ERROR: Failed to export ONNX model: {e}')


# --------------------------
# FINAL DEPLOYMENT ASSETS
# --------------------------
print('\n==============================================================================')
print('DEPLOYMENT ASSETS CREATED SUCCESSFULLY!')
print('1. VISITOR DATABASE (PKL): Ready for comparison logic on Jetson Nano.')
print('2. INFERENCE MODEL (ONNX): Ready for TensorRT optimization on Jetson Nano.')
print('==============================================================================')


--- 1. Starting Aggressive Dependency Management ---
Clearing conflicting packages...
Installing stable core dependencies...
✅ Installation complete. Loading modules...
Mounted at /content/drive

--- 3. File Cleanup & Aggressive Image Standardization ---
✅ Finished image standardization. 20 files processed successfully.

--- 4. Models Initialization ---

--- 5. Generating Embeddings (Using Manual CV2 Load) ---
❌ CRITICAL FINAL FAILURE processing /content/drive/MyDrive/IMAGE DATASET/yuvanew/YUVA4.jpeg: Could not infer dtype of numpy.uint8
❌ CRITICAL FINAL FAILURE processing /content/drive/MyDrive/IMAGE DATASET/yuvanew/YUVA5.jpeg: Could not infer dtype of numpy.uint8
❌ CRITICAL FINAL FAILURE processing /content/drive/MyDrive/IMAGE DATASET/yuvanew/YUVA3.jpeg: Could not infer dtype of numpy.uint8
❌ CRITICAL FINAL FAILURE processing /content/drive/MyDrive/IMAGE DATASET/yuvanew/YUVA1.jpeg: Could not infer dtype of numpy.uint8
❌ CRITICAL FINAL FAILURE processing /content/drive/MyDrive/IMAGE D

SystemExit: 1

In [ ]:
# ==============================================================================
# FINAL PIPELINE: OPENCV DNN FACE RECOGNITION ASSET GENERATION
# This script uses OpenCV's robust pre-trained models for detection and recognition.
# ==============================================================================
import subprocess
import sys
import os
import zipfile
import time
import requests
import io

# --- 1. Aggressive Dependency Management and Installation ---
print("--- 1. Starting Dependency Management and OpenCV Installation ---")

# 1.1. Aggressively uninstall conflicting packages (especially PyTorch components)
packages_to_uninstall = [
    "torch", "torchvision", "torchaudio", "facenet-pytorch", "onnx",
    "numpy", "opencv-python", "opencv-contrib-python", "opencv-python-headless"
]
try:
    subprocess.check_call([sys.executable, "-m", "pip", "uninstall"] + packages_to_uninstall + ["-y"],
                          stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
except:
    pass

# 1.2. Install stable, functional versions (NumPy, OpenCV)
print("Installing stable core dependencies...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy==1.26.4", "--quiet"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "opencv-python==4.9.0.80", "Pillow", "--quiet"])

print("✅ Installation complete. Loading modules...")

# --- 2. Module Imports and Config ---
import numpy as np
import pickle
import cv2
from PIL import Image
from google.colab import drive

# Define file paths (Adjust ZIP_FILE_PATH and ZIP_INNER_FOLDER_NAME)
ZIP_FILE_PATH = '/content/drive/MyDrive/visitor_data.zip' # Path to your uploaded ZIP file
EXTRACT_DIR = '/content/local_data'
# Assuming the zip extracts to a folder named 'CLEAN_DATASET' containing your visitor folders
DATA_DIR = os.path.join(EXTRACT_DIR, 'CLEAN_DATASET')
EMBEDDINGS_FILE = '/content/drive/MyDrive/visitor_embeddings_opencv.pkl'

# --- OpenCV Model Weights Download (Required for DNN) ---
# Face Detection Model (SSD MobileNet V1)
PROTOTXT = "deploy.prototxt"
MODEL = "res10_300x300_ssd_iter_140000.caffemodel"
if not os.path.exists(PROTOTXT):
    print("Downloading Caffe Face Detection Model...")
    # These are standard Caffe model links for face detection
    r = requests.get('https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt', allow_redirects=True)
    open(PROTOTXT, 'wb').write(r.content)
    r = requests.get('https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel', allow_redirects=True)
    open(MODEL, 'wb').write(r.content)
# Deep Learning Face Recognizer (Feature Extractor)
RECOGNIZER_MODEL = "res_model.t7"
if not os.path.exists(RECOGNIZER_MODEL):
    print("Downloading Face Recognition Model...")
    # This is a standard OpenCV deep learning face recognition model (ResNet based)
    r = requests.get('https://github.com/opencv/opencv_extra/raw/master/testdata/dnn/face_recognizer_v1_0/res_model.t7', allow_redirects=True)
    open(RECOGNIZER_MODEL, 'wb').write(r.content)
print("✅ OpenCV models downloaded.")


# --------------------------
# 3. PIPELINE STEP: DATA EXTRACTION (ZIP Loading)
# --------------------------
print("\n--- 3. Data Extraction (Bypassing Drive Mount) ---")
drive.mount('/content/drive', force_remount=True)

if not os.path.exists(DATA_DIR):
    try:
        os.makedirs(EXTRACT_DIR, exist_ok=True)
        with zipfile.ZipFile(ZIP_FILE_PATH, 'r') as zip_ref:
            zip_ref.extractall(EXTRACT_DIR)
        print(f"✅ Data extracted to local path: {DATA_DIR}")
    except Exception as e:
        print(f"❌ CRITICAL ERROR: Failed to extract ZIP file. Check if {ZIP_FILE_PATH} exists.")
        print(f"Error: {e}")
        sys.exit(1)


# --------------------------
# 4. PIPELINE STEP: MODELS INITIALIZATION
# --------------------------
print("\n--- 4. Models Initialization ---")
# Face Detector (Caffe)
face_detector = cv2.dnn.readNetFromCaffe(PROTOTXT, MODEL)
# Face Recognizer (Torch/ResNet)
face_recognizer = cv2.dnn.readNetFromTorch(RECOGNIZER_MODEL)
print('✅ OpenCV DNN Models initialized.')

# --------------------------
# 5. PIPELINE STEP: GENERATE EMBEDDINGS (THE CORE PROCESS)
# --------------------------
print("\n--- 5. Generating Embeddings ---")

# Utility function for face detection and extraction
def get_face_embedding(image_path, confidence_threshold=0.5):
    img = cv2.imread(image_path)
    (h, w) = img.shape[:2]

    # Create a 300x300 blob and run detection
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    face_detector.setInput(blob)
    detections = face_detector.forward()

    # Ensure at least one face is detected
    if detections.shape[2] > 0 and detections[0, 0, 0, 2] > confidence_threshold:
        i = 0 # Take the first and largest face detected
        confidence = detections[0, 0, i, 2]
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        # Extract the face ROI (Region of Interest)
        face = img[startY:endY, startX:endX]
        if face.shape[0] < 20 or face.shape[1] < 20: # Skip if face is too small
             return None

        # Generate a 96x96 blob for the recognition model
        face_blob = cv2.dnn.blobFromImage(face, 1.0/255, (96, 96), (0, 0, 0), swapRB=True, crop=False)

        # Generate the embedding (feature vector)
        face_recognizer.setInput(face_blob)
        embedding = face_recognizer.forward() # 128-dimensional vector

        return embedding.flatten()
    return None

# Manually collect paths and labels from the local, extracted folder
all_image_paths = []
all_labels = []
for root, _, files in os.walk(DATA_DIR):
    if root != DATA_DIR:
        label_name = os.path.basename(root)
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                all_image_paths.append(os.path.join(root, file))
                all_labels.append(label_name)

embeddings_raw = {label: [] for label in set(all_labels)}
success_count = 0

for img_path, name in zip(all_image_paths, all_labels):
    try:
        embedding = get_face_embedding(img_path)

        if embedding is not None:
            embeddings_raw[name].append(embedding)
            success_count += 1
        else:
            print(f'⚠️ Warning: No face detected in {name} image: {os.path.basename(img_path)}. Skipping.')
    except Exception as e:
        print(f'❌ CRITICAL FAILURE processing {img_path}: {e}')

print(f'\n✅ Finished generating raw embeddings. Total successful: {success_count}')
if success_count == 0:
    print("❌ Aborting export as no embeddings were created.")
    sys.exit(1)


# --------------------------
# 6. PIPELINE STEP: CREATE & SAVE VISITOR DATABASE (PKL)
# --------------------------
visitor_database = {}
for name, emb_list in embeddings_raw.items():
    if len(emb_list) > 0:
        mean_embedding = np.mean(emb_list, axis=0)
        # Normalization is crucial for Cosine Similarity
        visitor_database[name] = mean_embedding / np.linalg.norm(mean_embedding)

try:
    with open(EMBEDDINGS_FILE, 'wb') as f:
        pickle.dump(visitor_database, f)
    print(f'\n--- 6. Final Visitor Database (PKL) ---')
    print(f'Database saved to: {EMBEDDINGS_FILE}')
except Exception as e:
    print(f'❌ ERROR: Could not save PKL file to Drive: {e}')


# --------------------------
# 7. PIPELINE STEP: FINAL DEPLOYMENT ASSETS (No ONNX export for this model)
# --------------------------
# NOTE: OpenCV DNN models (Caffe/Torch) often perform well on the Jetson Nano
# without explicit ONNX/TensorRT conversion, relying on optimized JetPack libraries.
# The Jetson Nano script will load the .caffemodel, .prototxt, and .t7 files directly.

print('\n==============================================================================')
print('DEPLOYMENT ASSETS CREATED SUCCESSFULLY!')
print('The Jetson Nano requires 3 model files and 1 database file:')
print(f'1. VISITOR DATABASE (PKL): {EMBEDDINGS_FILE}')
print(f'2. FACE DETECTOR WEIGHTS: {MODEL}')
print(f'3. FACE DETECTOR ARCHITECTURE: {PROTOTXT}')
print(f'4. FACE RECOGNIZER MODEL: {RECOGNIZER_MODEL}')
print('Copy these four files to your Jetson Nano for real-time inference.')
print('==============================================================================')

--- 1. Starting Dependency Management and OpenCV Installation ---
Installing stable core dependencies...
✅ Installation complete. Loading modules...
✅ OpenCV models downloaded.

--- 3. Data Extraction (Bypassing Drive Mount) ---


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Mounted at /content/drive
❌ CRITICAL ERROR: Failed to extract ZIP file. Check if /content/drive/MyDrive/visitor_data.zip exists.
Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/visitor_data.zip'
Traceback (most recent call last):
  File "/tmp/ipython-input-1267009076.py", line 78, in <cell line: 0>
    with zipfile.ZipFile(ZIP_FILE_PATH, 'r') as zip_ref:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/zipfile/__init__.py", line 1352, in __init__
    self.fp = io.open(file, filemode)
              ^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/visitor_data.zip'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1267009076.py", line 84, in <cell line:

TypeError: object of type 'NoneType' has no len()